In [129]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pickle
import re
import itertools
import time
import pandas as pd
import numpy as np

url = "https://pokeassistant.com/movesets"
head = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

def get_moves(poke):
    name = re.sub(r'\s+', '', poke.select(".sethead")[0].text)
    moves = [re.search("[A-z][A-z -]+", s.text).group(0) for s in poke.select(".row")]
    return name, moves 

def get_pokes(url):
    req = Request(url, headers=head)
    page = urlopen(req).read()
    source = BeautifulSoup(page, "html.parser")
    pokes = source.select('.tier2')
    return(pokes)

In [84]:
pokes = get_pokes(url)

In [155]:
pokelist = [get_moves(p) for p in pokes]
pokedict = {p[0]: p[1] for p in pokelist}
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in pokedict.items() ]))
df = df.unstack().reset_index().drop(columns = "level_1")
df.columns = ["name", "move"]
df = df.dropna().reset_index(drop=True)

In [156]:
df.to_csv("movelist.csv", index=False)